In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time, os

In [2]:
fbref_query = "https://fbref.com/en/comps/9/history/Premier-League-Seasons"

In [3]:
def getAndParseURL(url):
    """
    This function takes a URL as input and returns a BeautifulSoup object.

    :param url: The URL to be fetched.
    :return: A BeautifulSoup object representing the parsed HTML content of the URL.
    """

    # Send a GET request to the specified URL.
    # The User-Agent header is set to mimic a web browser.
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})

    # Check if the request was successful (status code 200).
    if result.status_code == 200:

        # Parse the HTML content of the URL using BeautifulSoup.
        soup = BeautifulSoup(result.text, "html.parser")

        # Return the BeautifulSoup object.
        return soup

    else:
        # If the request was not successful, print an error message.
        print("Error: Unable to fetch URL. Status code:", result.status_code
              + ". Please check your internet connection or try again later.")

In [4]:
getAndParseURL(fbref_query)


<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202311031" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '

### From 1990 to 2023

In [17]:

# Define an empty list to store the URLs of the seasons
seasons = []

# Get the HTML content of the URL and parse it
html = getAndParseURL(fbref_query)

# Define the number of seasons to retrieve
season_count = 14

# Loop through the HTML content to find the URLs of the seasons
for season in html.findAll("td", {"data-stat": "league_name"}):
    # Append the URL of the season to the list
    seasons.append("https://fbref.com" + season.find("a").get("href"))
    
    # Decrement the count of seasons to retrieve
    season_count -= 1
    
    # If the count reaches 0, break out of the loop
    if season_count == 0:
        break

# Print the list of season URLs
seasons
#
#This code retrieves the URLs of the seasons from the given URL. It uses the BeautifulSoup library to parse the HTML content and find the URLs. The code stores the URLs in a list and prints the list..</s>

['https://fbref.com/en/comps/9/Premier-League-Stats',
 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier

In [6]:
data = requests.get(seasons[1])
current_df = pd.read_html(data.text, match="Squad Standard Stats")[0]

In [7]:
current_df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Squad               # Pl                Age   
0             Arsenal                 26               24.7   
1         Aston Villa                 26               27.0   
2         Bournemouth                 31               26.3   
3           Brentford                 25               26.2   
4            Brighton                 29               26.3   
5             Chelsea                 32               26.3   
6      Crystal Palace                 26               26.7   
7             Everton                 28               26.6   
8              Fulham                 29               28.2   
9        Leeds United                 29               25.3   
10     Leicester City                 28               26.1   
11          Liverpool                 28               27.4   
12    Manchester City                 24               27.2   
13     Manchester Utd                 26               26.9   
14      Newcastle Utd                 27               27.3   
15    Nott'ham Forest                 33               26.5   
16        Southampton                 36               24.7   
17          Tottenham                 29               27.6   
18           West Ham                 25               28.2   
19             Wolves                 32               26.7   

   Unnamed: 3_level_0 Playing Time                    Performance      ...  \
                 Poss           MP Starts   Min   90s         Gls Ast  ...   
0                59.3           38    418  3420  38.0          84  64  ...   
1                49.3           38    418  3420  38.0          49  35  ...   
2                40.4           38    418  3420  38.0          37  24  ...   
3                43.8           38    418  3420  38.0          56  36  ...   
4                60.2           38    418  3420  38.0          68  46  ...   
5                58.7           38    418  3420  38.0          37  27  ...   
6                46.3           38    418  3420  38.0          38  29  ...   
7                42.8           38    418  3420  38.0          32  24  ...   
8                48.8           38    418  3420  38.0          52  33  ...   
9                47.0           38    418  3420  38.0          45  31  ...   
10               47.7           38    418  3420  38.0          49  36  ...   
11               60.8           38    418  3420  38.0          71  58  ...   
12               64.7           38    418  3420  38.0          92  69  ...   
13               53.7           38    418  3420  38.0          56  42  ...   
14               52.3           38    418  3420  38.0          64  42  ...   
15               37.6           38    418  3420  38.0          36  22  ...   
16               44.5           38    418  3420  38.0          36  25  ...   
17               50.0           38    418  3420  38.0          68  46  ...   
18               42.1           38    418  3420  38.0          41  25  ...   
19               49.9           38    418  3420  38.0          28  12  ...   

   Per 90 Minutes                                                             
              Gls   Ast   G+A  G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0            2.21  1.68  3.89  2.13   3.82  1.89  1.41   3.31  1.82     3.23  
1            1.29  0.92  2.21  1.21   2.13  1.32  1.02   2.34  1.24     2.26  
2            0.97  0.63  1.61  0.97   1.61  1.02  0.75   1.76  1.02     1.76  
3            1.47  0.95  2.42  1.29   2.24  1.49  1.02   2.51  1.33     2.35  
4            1.79  1.21  3.00  1.63   2.84  1.93  1.37   3.30  1.81     3.19  
5            0.97  0.71  1.68  0.89   1.61  1.30  1.02   2.32  1.24     2.26  
6            1.00  0.76  1.76  0.97   1.74  1.03  0.76   1.80  0.97     1.73  
7            0.84  0.63  1.47  0.76   1.39  1.19  0.85   2.04  1.13     1.98  
8            1.37  0.87  2.24  1.24   2.11  1.22  0.75   1.97  1.03     1.78  
9            1.18  0.82  2.00 

In [8]:
def getTables(matchTable,seasons,table):
    """
    This function takes a matchTable string, a list of seasons, and an empty list table.
    It iterates through each season in the seasons list, making a GET request to the season URL.
    The function then parses the HTML response using BeautifulSoup and extracts the year from the HTML.
    It reads the table data from the HTML response into a pandas DataFrame and adds the year as a new column to the DataFrame.
    The DataFrame is then appended to the table list.
    """
    for season in seasons:
        try:
            # Make a GET request to the season URL
            data = requests.get(season)
            
            # Parse the HTML response using BeautifulSoup
            soup = getAndParseURL(season)
            
            # Extract the year from the HTML
            year = soup.find("div",{"class":"comps"}).h1.text.strip().split()[0]
            
            # Read the table data from the HTML response into a pandas DataFrame
            current_df = pd.read_html(data.text, match=matchTable)[0]
            
            # Add the year as a new column to the DataFrame
            current_df["season"] = year
            
            # Append the DataFrame to the table list
            table.append(current_df)
            
        except:
            # Print the season URL if an exception occurs
            print(season)
        
        # Sleep for 5 seconds to prevent overloading the server
        time.sleep(5)

In [10]:

# Create empty lists for each category
standart_stats = []
regular_season = []
shooting = []
passing = []
goal_shot_creation = []
defensive_action = []
possession = []

# Create a list of lists for easy access and manipulation
table_list = [standart_stats, regular_season, shooting, passing, goal_shot_creation, defensive_action, possession]

# Create a list of matching strings for each category
matching_list = ["Squad Standard Stats", "Regular season", "Squad Shooting", "Squad Passing", "Squad Goal and Shot Creation", "Squad Defensive Actions", "Squad Possession"]

# This code creates empty lists for each category of data and stores them in a list of lists called table_list.
# The matching_list contains strings that match the categories in table_list.
# This structure allows for easy access and manipulation of the data.

In [22]:
getTables("Squad Possession",seasons,possession)
possession_results = pd.concat(possession, axis=0, ignore_index=True)
possession_results.to_csv("possession.csv")

https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats
https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats
https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats
https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats
https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats
https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats
https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier-League-Stats


In [ ]:
# otomatik hale getir
#def get_write_data(tables,seasons,matchList): 

### --------------------

In [24]:
soup = getAndParseURL('https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats')

In [25]:
soup.find("div",{"class":"comps"}).h1.text.strip().split()[0]

'2021-2022'